<a href="https://colab.research.google.com/github/morettitommaso/sql-exercises/blob/main/edb01_montagna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caricamento dati nel DBMS

La cella qua sotto serve per per scaricare il database e interagire con il DBMS.

Se il DBMS semtte di funzionare, rieseguite la  cella di codice sottostante.

In [1]:
# Cella con il codice per il caricamento dei dati nel DBMS
# Per favore, non modificate il codice di questa cella
urldb='https://drive.google.com/uc?export=download&id=1Kc2bJtj1WHXO9o3yr-JsDdvJOIjpi8NX'
fn='db01interrogatorio.db'
pwd=''

import sqlite3
import urllib
import urllib.request
import zipfile
import os

import pandas as pd
pd.set_option('display.max_colwidth', None) # -1
pd.set_option('display.max_rows', 100)
from IPython.display import display, HTML

def downloadDBFile(urlFile, localFileName, pwd=''):
  # Dowloading the .zip file into the colab remote computation facility

  print('Downloading db file')
  try:
    urllib.urlretrieve(urlFile, localFileName) # Works in Python2
    print('Python2 detected')
  except AttributeError:
    print('Python 3 detected')
    with urllib.request.urlopen(urlFile) as response: # Works in Python3
      data = response.read()
      fd=open(localFileName,'wb')
      fd.write(data)
    fd.close()
  print('Download Done')
  if localFileName.endswith('.zip'):
    zipFileName = localFileName
    localFileName = zipFileName[:-4]
    print('Unzipping the file content')
    with zipfile.ZipFile(zipFileName, 'r') as zip_ref:
      zip_ref.extractall(localFileName)
      print('File %s extracted' % localFileName)
  elif localFileName.endswith('.pez'):
    zipFileName = localFileName
    localFileName = zipFileName[:-4]
    print('Unzipping the password protected file content')
    with zipfile.ZipFile(zipFileName, 'r') as zip_ref:
      #zip_ref.extractall(localFileName, pwd=bytes(pwd,'utf-8')) # it messes up the paths with password encrypted files
      zip_ref.extract(localFileName, pwd=bytes(pwd,'utf-8')) # localFileName: the name of the file in the .zip archive
      print('File %s extracted' % localFileName)

  localFileName = os.path.abspath(localFileName)
  print('File saved in '+localFileName)
  return localFileName
  print('Unzipping the password protected file content')


dbFileFullPath = downloadDBFile(urldb, fn, pwd=pwd)

def rq(query, threshold=100): # rq is for Run Query
    conn = sqlite3.connect(dbFileFullPath)
    if query.replace(' ','').replace('\n','') == '':
      return
    try:
        res = pd.read_sql(query,conn)
        conn.close()
        nrecords=res.shape[0]
        if nrecords<=threshold:
          display(res)
          print("%d records returned" % (nrecords) )
          #HTML("<b> %d records returned </b>" % (nrecords) )
          #return res

        else:
          display(res.iloc[:threshold,:])
          print("Too many records in the result, returned only the first %d" % (threshold))
          #return res.iloc[:threshold,:]

    except (pd.io.sql.DatabaseError, Exception)  as dbe: # (sqlite3.DatabaseError , sqlite3.OperationalError)
        print('Error')
        print(type(dbe))
        print(dbe)

Python 3 detected
Download Done
File saved in /content/db01interrogatorio.db


# EDB01 Giallo in montagna

Il **database interrogatorio** è composto da un’unica tabella di nome **dati**.

Siete in vacanza sulle piste da sci della valle XYZ ma purtroppo la polizia locale, guidata dall’ispettore Bianchi, ha chiuso le piste da sci. Il corpo di una persona morta è stato trovato vicino alle piste superiori in una zona boschiva.

L’ispettore Bianchi ha interrogato tutti i turisti nella zona e ha inserito tutti i dati raccolti in un database relazionale.

Purtroppo l'ispettore Bianchi non è molto bravo a interrogare i database, pertanto ha bisogno di aiuto e tu sei il miglior esperto di database disponibile sul posto.

Il database è composto da un’unica tabella dal nome “dati”. Una tupla della tabella riassume le informazioni raccolte dall’interrogatorio di una persona. Ogni tupla contiene i seguenti attributi:
*	ID: l’identificatore della tupla
*	DATA: la data in cui il sospettato è arrivato in albergo
*	NOME, COGNOME: il nome e cognome della persona interrogata
*	TARGA_AUTO: la targa dell’auto posseduta dalla persona
*	CITTA: la citta di provenienza della persona
*	PARTENZA, RITORNO: L’ora in cui la persona è rispettivamente uscita e rientrata nell’hotel per eseguire un’escursione nel giorno del delitto
*	CAPELLI: il colore dei capelli
*	MISURASCARPE: il numero (taglia) di scarpa della persona.

Nel database ci sono 75 tuple. Se appare una 76ma tupla con valori tutti nulli, trascuratela.

PS: nomi, luoghi, e tutte le altre informazioni contenute nel database sono inventati e non hanno alcuna attinenza con la realtà.

**Prova ad eseguire la cella di codice seguente.**

In [2]:
# Con il comando seguente potete vedere tutte le righe della tabella
rq("""
select * from dati;
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,1,2001-05-15,Ciro,AGOSTINI,COO-946,Milano,10:55:00,19:20:00,biondi,40
1,2,2001-05-15,Diego,ALBERTI,CHD-354,Napoli,08:20:00,19:05:00,neri,35
2,3,2001-05-15,Cecilia,BERETTA,KCV-836,Cosenza,07:55:00,18:35:00,rossi,36
3,4,2001-05-15,Erica,BERGAMASCHI,CKV-239,San Remo,07:55:00,18:35:00,castani,43
4,5,2001-05-15,Cristiano,BINDA,CJE-861,Venezia,10:50:00,20:55:00,castani,40
5,6,2001-05-15,Enrico,BONFANTI,VNZ-008,Ancona,09:30:00,19:30:00,neri,38
6,7,2001-05-15,Fabio,CERIANI,VNQ-867,Ancona,09:30:00,19:30:00,biondi,34
7,8,2001-05-15,Francesco,CERRATO,ZPO-912,Trieste,08:05:00,18:00:00,rossi,35
8,9,2001-05-15,Giordano,CREPALDI,NMO-986,Terni,08:10:00,18:00:00,castani,43
9,10,2001-05-15,Bianca,DANESI,CDB-460,Lecce,09:20:00,19:05:00,biondi,37


75 records returned


Dopo aver eseguito la cella di codice, qua sopra apparirà un risultato.

La prima colonna mostrerà il numero progressivo dei record restituiti.

Prestate attenzione all'ultimissima riga visualizzata, vi dice quanti record sono stati restituiti.

Se apparirà l'errore **NameError: name 'rq' is not defined**, (ri)eseguite la cella di codice che trovate nella sezione precedente
**Caricamento dati nel DBMS**.

In [3]:
# Negli esercizi successivi, per scrivere una query:
# 1) Copiate in una cella di codice il comando seguente;
# 2) Scrivete la query nella riga libera (tra le triple virgolette),
#    la query può essere scritta su più righe (potete andare a capo);
# 3) Eseguite la cella di codice.
rq("""
select * from dati
where misurascarpe > 45;
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,59,2001-07-07,Tommaso,SALERNO,ZTI-906,Bergamo,08:20:00,19:05:00,neri,46


1 records returned


**Riepilogo sintassi query SQL**

SELECT Attributo1, Attributo2, ..., *

FROM Tabella1, Tabella2, …, TabellaN

WHERE CondizioniSemplici

ORDER BY ListaAttributiDiOrdinamento

;


#Per aiutare l’ispettore Bianchi, svolgete le seguenti operazioni:

Se vi è comodo potete tenere aperta la
[tabella dati](https://docs.google.com/spreadsheets/d/1UPgw9Adnz_RUd2xxlaDmjkXB8-1fftc-v7dnL9w9QLQ/preview) aprendola in un'altra finestra del browser.
<!--https://docs.google.com/spreadsheets/d/1UPgw9Adnz_RUd2xxlaDmjkXB8-1fftc-v7dnL9w9QLQ/edit?usp=sharing -->

## (EDB01) 1
Visualizzate l’elenco di tutti i sospettati.

Se vi è stato spiegato come effettuare l’ordinamento, ordinate i risultati in base al cognome.


In [4]:
rq("""
select * from dati
order by dati.cognome
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,1,2001-05-15,Ciro,AGOSTINI,COO-946,Milano,10:55:00,19:20:00,biondi,40
1,2,2001-05-15,Diego,ALBERTI,CHD-354,Napoli,08:20:00,19:05:00,neri,35
2,36,2001-05-22,Camilla,ALBERTI,COU-824,Aosta,09:20:00,19:50:00,rossi,42
3,3,2001-05-15,Cecilia,BERETTA,KCV-836,Cosenza,07:55:00,18:35:00,rossi,36
4,37,2001-05-23,Salvatore,BERETTA,CHD-354,Napoli,10:05:00,18:20:00,rossi,36
5,4,2001-05-15,Erica,BERGAMASCHI,CKV-239,San Remo,07:55:00,18:35:00,castani,43
6,38,2001-05-24,Diego,BERGAMASCHI,KCV-836,Cosenza,08:35:00,16:50:00,castani,38
7,5,2001-05-15,Cristiano,BINDA,CJE-861,Venezia,10:50:00,20:55:00,castani,40
8,39,2001-05-24,Cecilia,BINDA,CKV-239,San Remo,09:20:00,19:50:00,neri,42
9,6,2001-05-15,Enrico,BONFANTI,VNZ-008,Ancona,09:30:00,19:30:00,neri,38


75 records returned


## (EDB01) 2
L’ispettore Bianchi ha avuto alcuni suggerimenti da un informatore ...

#### (EDB01) 2.a

*  ... Il colpevole dovrebbe essere uscito dall’albergo alle 10:50.
* Scrivete una query che mostri tutte le persone che sono uscite dall’albergo alle 10:50 (condizione: partenza='10:50:00', occorre indicare anche i secondi). In generale, se fate copia e incolla da una qualsiasi fonte, prestate attenzione al tipo di apice singolo usato. Dei seguenti caratteri  ’, ‘, ’,  ' (dal punto di vista tipografico, sono caratteri diversi), solamente ' (l'apostrofo) viene riconosciuto correttamente dall'interprete SQL.

*  Nota: 5 persone nel risultato.

In [5]:
rq("""
select *
from dati
where partenza='10:50:00';
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,5,2001-05-15,Cristiano,BINDA,CJE-861,Venezia,10:50:00,20:55:00,castani,40
1,23,2001-05-18,Pasquale,MAGGIONI,CJE-861,Venezia,10:50:00,20:55:00,rossi,44
2,44,2001-05-25,Luigi,FILIPPINI,LLP-990,Trieste,10:50:00,20:55:00,biondi,36
3,56,2001-05-29,Ciro,IULIANO,CHD-354,Napoli,10:50:00,20:55:00,biondi,38
4,62,2001-07-07,William,SIGNORINI,VIP-994,Perugia,10:50:00,20:55:00,castani,40


5 records returned


### (EDB01) 2.b
*  ... Il colpevole dovrebbe  avere i capelli biondi.
* Scrivete una query che mostri tutte le persone con i capelli biondi (indipendentemente dall’ora in cui sono uscite)

* Nota: 21 persone nel risultato.

In [6]:
rq("""
select *
from dati
where dati.capelli = "biondi";
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,1,2001-05-15,Ciro,AGOSTINI,COO-946,Milano,10:55:00,19:20:00,biondi,40
1,7,2001-05-15,Fabio,CERIANI,VNQ-867,Ancona,09:30:00,19:30:00,biondi,34
2,10,2001-05-15,Bianca,DANESI,CDB-460,Lecce,09:20:00,19:05:00,biondi,37
3,13,2001-05-16,Chiara,FARINELLI,LLP-990,Trieste,10:00:00,18:50:00,biondi,43
4,19,2001-05-18,Ciro,GRANATA,COO-946,Milano,10:55:00,19:20:00,biondi,36
5,22,2001-05-18,Agata,LOI,CKV-239,San Remo,07:55:00,18:35:00,biondi,40
6,28,2001-05-20,Renato,OTTAVIANI,CDB-460,Lecce,09:20:00,19:05:00,biondi,42
7,29,2001-05-20,Giulia,PADOVANI,ZQR-643,Vicenza,09:20:00,19:05:00,biondi,43
8,33,2001-05-21,Antonio,QUARTA,XXZ-175,Pescara,09:55:00,18:45:00,biondi,40
9,35,2001-05-22,Nicola,RICCARDI,CPO-924,Vercelli,08:35:00,16:50:00,biondi,37


21 records returned


### (EDB01) 2.c
* ... Il colpevole dovrebbe  avere un numero di scarpa pari a 44.
* Scrivete una query che mostri tutte le persone con scarpe di numero 44  (non considerate le condizioni precedenti).

* Nota: 3 persone nel risultato.

In [9]:
rq("""
select *
from dati
where dati.misurascarpe = 44;
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,11,2001-05-16,Guido,ESPOSITO,ZQR-643,Vicenza,09:20:00,19:05:00,rossi,44
1,23,2001-05-18,Pasquale,MAGGIONI,CJE-861,Venezia,10:50:00,20:55:00,rossi,44
2,60,2001-07-07,Valentino,SCOTTI,PGR-822,Firenze,07:55:00,18:35:00,biondi,44


3 records returned


### (EDB01) 2.d

* Scrivete una query che mostri le persone che che sono uscite dall’albergo alle 10:50 e hanno i capelli biondi.

* Nota: 2 persone nel risultato.


In [10]:
rq("""
select *
from dati
where (partenza='10:50:00' and capelli = "biondi");
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,44,2001-05-25,Luigi,FILIPPINI,LLP-990,Trieste,10:50:00,20:55:00,biondi,36
1,56,2001-05-29,Ciro,IULIANO,CHD-354,Napoli,10:50:00,20:55:00,biondi,38


2 records returned


### (EDB01) 2.e
* Scrivete una query che mostri le persone che hanno contemporaneamente  le caratteristiche indicate nei punti .a .b .c (tralasciate il punto .d)

* Nota: risultato nullo.  Secondo voi l’informatore è attendibile?


In [17]:
rq("""
select *
from dati
where (dati.partenza='10:50:00' and dati.capelli = "biondi" and dati.misurascarpe = 44);
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe


0 records returned


## (EDB01) 3
Le indagini vengono riprese.

L’ispettore vuole intervistare di nuovo alcuni turisti.

Scrivete delle query per aiutare l’ispettore Bianchi ad individuare le persone descritte nei punti seguenti:

### (EDB01) 3.a
* ... una persona con i capelli neri che è stata vista uscire alle 08:10.

* Nota: 1 persona di nome Stefania nel risultato.


In [12]:
rq("""
select *
from dati
where (partenza = "08:10:00" and capelli = "neri");
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,66,2001-07-07,Stefania,TARTAGLIA,CDB-460,Cesena,08:10:00,18:00:00,neri,39


1 records returned


### (EDB01) 3.b
* ... delle persone con i capelli neri viste rientrare in albergo alle 19:05. Ignorate le condizioni del punto precedente.

* Nota: 4 persone nel risultato



In [18]:
rq("""
select *
from dati
where (dati.ritorno == "19:05:00" and dati.capelli == "neri");
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,2,2001-05-15,Diego,ALBERTI,CHD-354,Napoli,08:20:00,19:05:00,neri,35
1,59,2001-07-07,Tommaso,SALERNO,ZTI-906,Bergamo,08:20:00,19:05:00,neri,46
2,67,2001-07-07,Maddalena,TONINI,ZQR-643,Bologna,09:20:00,19:05:00,neri,41
3,68,2001-07-07,Corrado,TOSCANO,COO-946,Bologna,09:20:00,19:05:00,neri,43


4 records returned


### (EDB01) 3.c
* ... una persona con i capelli biondi e con cognome Agostini  che è stata vista parlare con la vittima i giorni precedenti al delitto. Ignorate le condizioni dei punti precedenti.

* Nota: id=1 nel risultato.


In [25]:
rq("""
select *
from dati
where (dati.capelli == "biondi" AND dati.cognome == "AGOSTINI");
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,1,2001-05-15,Ciro,AGOSTINI,COO-946,Milano,10:55:00,19:20:00,biondi,40


1 records returned


## (EDB01) 4
L'ispettore ha raccolto importanti informazioni nei giorni trascorsi, anche grazie alle ultime persone interrogate. Tra le informazioni raccolte, le più importanti sono:

### (EDB01) 4.a
* Un gruppo di alpinisti ha segnalato che la vittima e un’altra persona con i capelli castani sono state viste assieme sulla montagna alle 15:30 del pomeriggio (quindi devono essere partite prima di tale ora).
* Chi potrebbe essere l’accompagnatore della vittima?
* Nota: 21 persone nel risultato.

In [26]:
rq("""
select *
from dati
where (dati.partenza < "15:30;00" AND dati.capelli = "castani");
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,4,2001-05-15,Erica,BERGAMASCHI,CKV-239,San Remo,07:55:00,18:35:00,castani,43
1,5,2001-05-15,Cristiano,BINDA,CJE-861,Venezia,10:50:00,20:55:00,castani,40
2,9,2001-05-15,Giordano,CREPALDI,NMO-986,Terni,08:10:00,18:00:00,castani,43
3,14,2001-05-16,Maurizio,FILIPPINI,CBW-966,Latina,10:35:00,19:20:00,castani,39
4,20,2001-05-18,Diego,IACONO,CHD-354,Napoli,08:20:00,19:05:00,castani,38
5,26,2001-05-20,Paolo,NARDI,ZPO-912,Trieste,08:05:00,18:00:00,castani,41
6,30,2001-05-21,Luigi,PANIZZA,MCP-919,Bolzano,10:05:00,19:30:00,castani,39
7,32,2001-05-21,Greta,PELLEGRINO,CBW-966,Latina,10:35:00,19:20:00,castani,34
8,38,2001-05-24,Diego,BERGAMASCHI,KCV-836,Cosenza,08:35:00,16:50:00,castani,38
9,41,2001-05-24,Cristiano,ESPOSITO,VNZ-008,Ancona,08:20:00,19:05:00,castani,40


21 records returned


### (EDB01) 4.b
* Gli alpinisti non ricordavano con precisione chi fosse la persona, ma erano certi che si trattasse di qualcuno arrivato in hotel il 7 luglio (data='2001-07-07').
* Chi può essere?
* Nota: 8 persone nel risultato considerando gli indizi dei punti a) e b) congiuntamente.

In [27]:
rq("""
select *
from dati
where (dati.partenza < "15:30;00" AND dati.capelli = "castani" AND dati.data='2001-07-07');
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,58,2001-07-07,Tiziano,ROMEO,IYE-437,Varese,10:55:00,19:20:00,castani,37
1,62,2001-07-07,William,SIGNORINI,VIP-994,Perugia,10:50:00,20:55:00,castani,40
2,64,2001-07-07,Miriam,SPREAFICO,EBI-143,Pesaro,09:30:00,19:30:00,castani,42
3,65,2001-07-07,Alberto,STEFANINI,AJE-313,Rimini,08:05:00,18:00:00,castani,45
4,69,2001-07-07,Eva,URBANO,CHD-354,Matera,10:05:00,19:30:00,castani,41
5,70,2001-07-07,Edoardo,VENTURA,CBW-966,Trieste,10:00:00,18:50:00,castani,40
6,72,2001-07-07,Fabrizio,POZZI,ZIP-756,Lecco,09:55:00,18:45:00,castani,42
7,75,2001-07-07,Michela,ZUCCHI,CCS-284,Sassari,09:20:00,19:50:00,castani,38


8 records returned


### (EDB01) 4.c
* Sono state trovate impronte di scarpe di taglia 40 vicino al luogo del delitto. L’assassino deve avere scarpe di taglia 40 dato che la vittima ha 36 come misura.
* Chi sono i sospettati?
* Nota: 2 persone nel risultato considerando gli indizi dei punti a), b) e c) congiuntamente.

In [28]:
rq("""
select *
from dati
where (dati.partenza < "15:30;00"
AND dati.capelli = "castani"
AND dati.data='2001-07-07'
AND dati.misurascarpe = 40);
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,62,2001-07-07,William,SIGNORINI,VIP-994,Perugia,10:50:00,20:55:00,castani,40
1,70,2001-07-07,Edoardo,VENTURA,CBW-966,Trieste,10:00:00,18:50:00,castani,40


2 records returned


### (EDB01) 4.d
* d.	Sfortunatamente l’ispettore Bianchi non dispone di informazioni sufficienti per accusare ufficialmente qualcuno. Tuttavia, dopo approfondite ricerche, un nuovo testimone si fa avanti, il quale ricorda di aver visto un’auto ai piedi della montagna. La targa iniziava con CBW (il testimone non ricorda il nome completo della città).

  *	Nota: si suggerisce di non cercare di filtrare ulteriormentee i risultati andando a cercare la sottostringa 'CBW' all’interno dell’attributo targa (tra l’altro non vi è stato spiegato a lezione) ma di guardare i dati.
  * Se vi è stato spiegato a lezione, ordinate i risultati del punto precedente in base alla targa. Chi è il sospettato?

  *	Purtroppo la polizia scopre che Edoardo Ventura ha un alibi. Non può essere lui l'assassino!

In [30]:
rq("""
select *
from dati
where (dati.partenza < "15:30;00"
AND dati.capelli = "castani"
AND dati.data='2001-07-07'
AND dati.misurascarpe = 40)
ORDER BY dati.targa_auto;
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,70,2001-07-07,Edoardo,VENTURA,CBW-966,Trieste,10:00:00,18:50:00,castani,40
1,62,2001-07-07,William,SIGNORINI,VIP-994,Perugia,10:50:00,20:55:00,castani,40


2 records returned


## (EDB01) 5
* Ulteriori indagini rivelano che alcune delle ultime dichiarazioni dei testimoni non sono attendibili.
* A seguito di un appello televisivo sono usciti due nuovi testimoni i quali hanno fornito informazioni decisive. Le testimonianze fornite sono le seguenti:

### (EDB01) 5.a
* "... Ho visto una donna partire esattamente alle ore 09.30 dall’albergo in compagnia della vittima. Si tratta di una delle persone arrivate in hotel il 7 luglio 2001. Ho notato che le sue scarpe erano più grandi delle mie, io porto scarpe di taglia 36. Sono andata a fare una passeggiata e sono rientrata in albergo alle 19.30 e ho visto rientrare la signora alla stessa ora da sola. Lei aveva i capelli di un colore tra il rosso e il castano, non ricordo. ".

* Chi sono i possibili sospettati?

* Avvertenze: se nella condizione scriverete capelli='rossi' and 'castani', ciò non è corretto. Occorre invece scrivere la condizione per esteso capelli='rossi' and capelli='castani'.

* Infine, valutate attentamente il funzionamento degli operatori AND ed OR.

* Nota: 2 persone nei risultati.

In [32]:
rq("""
select *
from dati
where (dati.partenza = "09:30:00"
AND dati.data = "2001-07-07"
AND dati.misurascarpe > 36
AND dati.ritorno == "19:30:00"
AND (dati.capelli == "rossi" or dati.capelli == "castani"));
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,63,2001-07-07,Anna,SPADONI,FKI-892,Pisa,09:30:00,19:30:00,rossi,40
1,64,2001-07-07,Miriam,SPREAFICO,EBI-143,Pesaro,09:30:00,19:30:00,castani,42


2 records returned


### (EDB01) 5.b
* "... Sì, il giorno del delitto ho visto la vittima partire per un’escursione con una donna. Poco prima di partire, la signora ha recuperato un paio di scarponi da una valigia in una macchina. Una scritta nel porta targa dell'auto faceva riferimento alla provincia di Pisa o Pesaro non ricordo. Sulla valigia ricordo che erano incise le iniziali A. S.".
* Chi è l’assassino combinando gli indizi dei punti a) e b)?
* NB: viste le dimensioni dei risultati, non scrivete condizioni aggiuntive sulle iniziali del nome.

In [ ]:
rq("""

""")

### (EDB01) 5.c

Questo è un esercizio diverso dai precedenti e serve per fissare l'uso dei connettori logici e delle parentesi.

* Secondo voi le query seguenti producono lo stesso risultato oppure no? Eseguite le query seguenti nell'ordine indicato, controllate per ognuna il numero di tuple restituito.

  1. select * from dati where capelli='rossi' or capelli='castani' and partenza='09:30:00' ;

  2. select * from dati where partenza='09:30:00' and capelli='rossi' or capelli='castani' ;

  3. select * from dati where partenza='09:30:00' and (capelli='rossi' or capelli='castani') ;

Perché secondo voi le query non producono lo stesso risultato?

In [33]:
# Query 1
rq("""
select * from dati where capelli='rossi' or capelli='castani' and partenza='09:30:00' ;
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,3,2001-05-15,Cecilia,BERETTA,KCV-836,Cosenza,07:55:00,18:35:00,rossi,36
1,8,2001-05-15,Francesco,CERRATO,ZPO-912,Trieste,08:05:00,18:00:00,rossi,35
2,11,2001-05-16,Guido,ESPOSITO,ZQR-643,Vicenza,09:20:00,19:05:00,rossi,44
3,17,2001-05-17,Laura,GIACOBBE,CPO-924,Vercelli,08:35:00,16:50:00,rossi,34
4,18,2001-05-17,Michele,GIAQUINTO,COU-824,Aosta,09:20:00,19:50:00,rossi,42
5,23,2001-05-18,Pasquale,MAGGIONI,CJE-861,Venezia,10:50:00,20:55:00,rossi,44
6,25,2001-05-19,Claudia,MARANGONI,VNQ-867,Ancona,09:30:00,19:30:00,rossi,37
7,27,2001-05-20,Ginevra,ORSINI,NMO-986,Terni,08:10:00,18:00:00,rossi,40
8,34,2001-05-21,Maria,RAGAZZI,ZIP-756,Torino,10:05:00,18:20:00,rossi,38
9,36,2001-05-22,Camilla,ALBERTI,COU-824,Aosta,09:20:00,19:50:00,rossi,42


17 records returned


In [34]:
# Query 2
rq("""
select * from dati where partenza='09:30:00' and capelli='rossi' or capelli='castani' ;
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,4,2001-05-15,Erica,BERGAMASCHI,CKV-239,San Remo,07:55:00,18:35:00,castani,43
1,5,2001-05-15,Cristiano,BINDA,CJE-861,Venezia,10:50:00,20:55:00,castani,40
2,9,2001-05-15,Giordano,CREPALDI,NMO-986,Terni,08:10:00,18:00:00,castani,43
3,14,2001-05-16,Maurizio,FILIPPINI,CBW-966,Latina,10:35:00,19:20:00,castani,39
4,20,2001-05-18,Diego,IACONO,CHD-354,Napoli,08:20:00,19:05:00,castani,38
5,25,2001-05-19,Claudia,MARANGONI,VNQ-867,Ancona,09:30:00,19:30:00,rossi,37
6,26,2001-05-20,Paolo,NARDI,ZPO-912,Trieste,08:05:00,18:00:00,castani,41
7,30,2001-05-21,Luigi,PANIZZA,MCP-919,Bolzano,10:05:00,19:30:00,castani,39
8,32,2001-05-21,Greta,PELLEGRINO,CBW-966,Latina,10:35:00,19:20:00,castani,34
9,38,2001-05-24,Diego,BERGAMASCHI,KCV-836,Cosenza,08:35:00,16:50:00,castani,38


23 records returned


In [35]:
# Query 3
rq("""
select * from dati where partenza='09:30:00' and (capelli='rossi' or capelli='castani') ;
""")

,id,data,nome,cognome,targa_auto,citta,partenza,ritorno,capelli,misurascarpe
0,25,2001-05-19,Claudia,MARANGONI,VNQ-867,Ancona,09:30:00,19:30:00,rossi,37
1,45,2001-05-25,Chiara,FRIGERIO,CBW-966,Latina,09:30:00,19:30:00,castani,35
2,63,2001-07-07,Anna,SPADONI,FKI-892,Pisa,09:30:00,19:30:00,rossi,40
3,64,2001-07-07,Miriam,SPREAFICO,EBI-143,Pesaro,09:30:00,19:30:00,castani,42


4 records returned


### (EDB01) 5.d

* Alla luce di quanto emerso nel punto 5.c, controllate se avete scritto correttamente le query dei punti 5.a e 5.b